In [1]:
import requests
import pandas as pd
import math
from time import sleep
import json

In [2]:
# API URL
api_url = "https://exoplanetarchive.ipac.caltech.edu/TAP/sync?query=select+*+from+pscomppars&format=json"

In [3]:
# Make get request to the API
response = requests.get(api_url)
page = response
while page == response:
    try:
        page = response
        break
    except:
        print("Connection refused by the server..")
        print("Let me sleep for 5 seconds")
        print("ZZzzzz...")
        time.sleep(5)
        print("Was a nice sleep, now let me continue...")
        continue

In [4]:
# Save data to a JSON file with indentation for readability
data = response.json()
with open('static/json/exoplanet_data.json', 'w') as outfile: 
    json.dump(data, outfile, indent=2)

In [5]:
# Check if the request was successful (status code 200?)
Planet_Name = []
Discovered_Year = []
Orbital_Period_days = []
SemiMajor_Axis_AU = []
Mass = []
Stellar_Luminosity = []
ra = []
dec = []
radius = []
density = []
Spectral_Type = []
Temperature = []

if response.status_code == 200:
    # Parse JSON response
    data = response.json()
    # Test to print the first few records
    for entry in data:
        Planet_Name.append(entry.get("pl_name"))
        Discovered_Year.append(entry.get("disc_year"))
        Orbital_Period_days.append(entry.get("pl_orbper"))
        SemiMajor_Axis_AU.append(entry.get("pl_orbsmax"))
        Spectral_Type.append(entry.get("st_spectype"))
        Mass.append(entry.get("pl_bmasse"))
        Temperature.append(entry.get("st_teff"))
        Stellar_Luminosity.append(entry.get("st_lum")) 
        ra.append(entry.get("ra"))       
        dec.append(entry.get("dec"))
        radius.append(entry.get("pl_rade")) 
        density.append(entry.get("pl_dens"))
else:
    print("Error:", response.status_code)


In [6]:
# Create pandas dataframe
exoplanet_data = pd.DataFrame({'Planet Name':Planet_Name,'Discovery Year':Discovered_Year,'Orbital Period (days)':Orbital_Period_days,'SemiMajor Axis (AU)':SemiMajor_Axis_AU, 'Spectral_Type':Spectral_Type, 'Mass (Earth Mass)':Mass,'Temperature':Temperature,'Stellar_Luminosity':Stellar_Luminosity,'Right Ascension (RA)':ra,'Declination (Dec)':dec,'radius':radius,'density':density})
exoplanet_data = exoplanet_data.dropna()

# Count dataframe rows after dropping null values
print(len(exoplanet_data))

# create calculations for habitable zone
exoplanet_data['Stellar_Luminosity'] = 10**(exoplanet_data['Stellar_Luminosity'])
exoplanet_data['Temperature'] = exoplanet_data['Temperature']-273.15
exoplanet_data.head(10)

1753


Planet Name  Discovery Year  Orbital Period (days)  SemiMajor Axis (AU)  \
5    TOI-1260 c            2021               7.493134              0.06570   
6   HD 149143 b            2005               4.071820              0.05300   
7   HD 210702 b            2007             354.100000              1.14800   
8   HIP 12961 b            2010              57.435000              0.25000   
9    HD 11755 b            2015             433.700010              1.09000   
11  HD 115954 b            2021            3700.000000              5.00000   
12  HIP 97233 b            2014            1062.265060              2.49100   
13   HD 95872 b            2015            4375.000000              5.15000   
15    WASP-49 b            2012               2.781740              0.03790   
16    WASP-56 b            2012               4.617100              0.05614   

   Spectral_Type  Mass (Earth Mass)  Temperature  Stellar_Luminosity  \
5           K6 V           13.20000      3953.85            0.129122   
6             G0          422.71390      5582.85            2.238721   
7         K1 III          574.63664      4677.85           14.092888   
8             M0          114.41880      3627.85            0.099312   
9             G5         1789.38290      4038.85          147.570653   
11          G0 V         2685.65004      5683.85            3.026913   
12      K0/1 III         6098.17365      4746.85           15.995580   
13          K0 V         1188.68420      5038.85            0.572796   
15          G6 V          117.59710      5326.85            0.885116   
16            G6          143.02350      5326.85            1.267652   

    Right Ascension (RA)  Declination (Dec)  radius  density  
5             157.144071          65.854199   2.760    3.450  
6             248.212671           2.084455  13.600    0.923  
7             332.963864          16.040481  13.400    1.310  
8              41.680075         -23.086005  13.200    0.273  
9              29.707836          73.152137  12.800    4.690  
11            199.985369          38.369138  12.600    7.380  
12            296.422121          -0.696879  12.100   18.900  
13            165.865880         -22.094330  13.000    2.970  
15             91.089719         -16.965390  12.442    0.288  
16            183.366040          23.055687  10.536    0.583

In [7]:
# Find Habitable zone
lum = exoplanet_data['Stellar_Luminosity']
divisor_ri = 1.1
exoplanet_data['Ri (AU)'] = [math.sqrt(abs(x)/divisor_ri) for x in lum]
divisor_ro = 0.53
exoplanet_data['Ro (AU)'] = [math.sqrt(abs(x)/divisor_ro) for x in lum]
exoplanet_data['Habitable Zone'] = ['Yes' if exoplanet_data.iloc[x,12]<exoplanet_data.iloc[x,3] and exoplanet_data.iloc[x,13]>exoplanet_data.iloc[x,3] else 'No' for x in range(len(exoplanet_data))]


In [8]:
# Convert data format
exoplanet_data['Planet Name'] = exoplanet_data['Planet Name'].astype('string')
exoplanet_data['Discovery Year'] = exoplanet_data['Discovery Year'].astype('int64')
exoplanet_data['Orbital Period (days)'] = exoplanet_data['Orbital Period (days)'].astype('int64')
exoplanet_data['SemiMajor Axis (AU)'] = exoplanet_data['SemiMajor Axis (AU)'].astype('int64')
exoplanet_data['Spectral_Type'] = exoplanet_data['Spectral_Type'].astype('string')
exoplanet_data['Mass (Earth Mass)'] = exoplanet_data['Mass (Earth Mass)'].astype('int64')
exoplanet_data['Temperature'] = exoplanet_data['Temperature'].astype('int64')
exoplanet_data['Stellar_Luminosity'] = exoplanet_data['Stellar_Luminosity'].astype('int64')
exoplanet_data['Right Ascension (RA)'] = exoplanet_data['Right Ascension (RA)'].astype('int64')
exoplanet_data['Declination (Dec)'] = exoplanet_data['Declination (Dec)'].astype('int64')
exoplanet_data['radius'] = exoplanet_data['radius'].astype('int64')
exoplanet_data['density'] = exoplanet_data['density'].astype('int64')
exoplanet_data['Ri (AU)'] = exoplanet_data['Ri (AU)'].astype('float')
exoplanet_data['Ro (AU)'] = exoplanet_data['Ro (AU)'].astype('float')
exoplanet_data['Habitable Zone'] = exoplanet_data['Habitable Zone'].astype('string')


In [9]:
# Exoport to csv
exoplanet_data.to_csv('exoplanetdata.csv',encoding = "utf-8", index = False, header = True)
exoplanet_data.head()

Planet Name  Discovery Year  Orbital Period (days)  SemiMajor Axis (AU)  \
5   TOI-1260 c            2021                      7                    0   
6  HD 149143 b            2005                      4                    0   
7  HD 210702 b            2007                    354                    1   
8  HIP 12961 b            2010                     57                    0   
9   HD 11755 b            2015                    433                    1   

  Spectral_Type  Mass (Earth Mass)  Temperature  Stellar_Luminosity  \
5          K6 V                 13         3953                   0   
6            G0                422         5582                   2   
7        K1 III                574         4677                  14   
8            M0                114         3627                   0   
9            G5               1789         4038                 147   

   Right Ascension (RA)  Declination (Dec)  radius  density    Ri (AU)  \
5                   157                 65       2        3   0.342613   
6                   248                  2      13        0   1.426605   
7                   332                 16      13        1   3.579346   
8                    41                -23      13        0   0.300472   
9                    29                 73      12        4  11.582536   

     Ro (AU) Habitable Zone  
5   0.493585             No  
6   2.055238             No  
7   5.156584             No  
8   0.432875             No  
9  16.686378             No

In [10]:
# Find different data types in dataframe
exoplanet_data.dtypes

Planet Name               string
Discovery Year             int64
Orbital Period (days)      int64
SemiMajor Axis (AU)        int64
Spectral_Type             string
Mass (Earth Mass)          int64
Temperature                int64
Stellar_Luminosity         int64
Right Ascension (RA)       int64
Declination (Dec)          int64
radius                     int64
density                    int64
Ri (AU)                  float64
Ro (AU)                  float64
Habitable Zone            string
dtype: object

In [11]:
list = []
for x in range(len(exoplanet_data)):
    if exoplanet_data.iloc[x,14] == "Yes":
        list.append(exoplanet_data.iloc[x,0])
print(list)


['GJ 3293 d', 'GJ 180 c', 'HD 136118 b', 'GJ 433 d', 'HD 125612 b', 'HD 4732 c', 'GJ 687 b', 'HD 191939 g', 'HD 210277 b', 'HD 10180 g', 'HD 147379 b', 'HD 111998 b', 'HD 165155 b', 'TOI-700 d', 'BD+45 564 b', '55 Cnc f', 'Kepler-22 b', 'HD 17674 b', 'HD 175167 b', 'HD 190228 b', 'HD 99109 b', 'HD 213240 b', 'BD+14 4559 b', 'GJ 1002 b', 'HD 16175 b', 'GJ 96 b', 'HD 142415 b', 'HD 100777 b', 'HD 13167 b', 'HD 159868 b', 'HD 10697 b', 'HIP 56640 b', 'HD 108874 b', 'HD 188015 b', 'HD 18015 b', 'HD 4113 b', 'HD 218566 b', 'HD 221287 b', 'HD 222582 b', 'Kepler-1661 b', 'TOI-2257 b', 'GJ 273 b', 'GJ 667 C f', 'GJ 667 C c', 'HD 181720 b', 'HD 20782 b', 'HD 13908 c', 'HD 5319 b', 'HD 564 b', 'HD 183263 b', 'HD 147513 b', 'HD 114729 b', 'HD 28185 b', 'HD 125390 b', 'HD 38529 c', 'HD 145934 b', 'HD 63765 b', 'Kepler-155 c', 'HD 216435 b', 'GJ 3293 b', 'KELT-6 c', 'HD 103891 b', 'GJ 876 c', 'HD 23079 b', 'HD 40307 g', 'HD 86264 b', 'HD 137388 b', 'GJ 1061 d', 'HD 43197 b', 'ups And d', 'BD+55 362

In [12]:
# Check if the planets found in habitable zone include any of the exoplanets that are confirmed to be potentially habitable
plan = []

exoplanet_names = ['Proxima Cen b', 'TRAPPIST-1e', 'TRAPPIST-1f', 'TRAPPIST-1g', 'Kepler-452 b', 'LHS 1140 b','Earth','Kepler-186 f','Ross 128 b','GJ 667 C c']
for x in list:
    if x in exoplanet_names:
        plan.append(x)
print(plan)



['GJ 667 C c', 'Proxima Cen b']


In [13]:

import sqlite3

# Read CSV data into a pandas DataFrame
csv_file = 'exoplanetdata.csv'
data = pd.read_csv(csv_file)

# Define SQLite database file
db_file = 'exoplanet.sqlite'

# Create a connection to the SQLite database
conn = sqlite3.connect(db_file)

# Convert DataFrame to SQLite table
data.to_sql('exoplanet_table', conn, if_exists='replace', index=False)

# Close the database connection
conn.close()